# SONDEADJUST.C1 Plots

[Click here](https://www.arm.gov/capabilities/vaps/sondeadjust) for more information about this vap.

In [ ]:
%matplotlib widget
import ipywidgets as widgets

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from datetime import datetime

import act
import xarray as xr

# Data archive directory
DATA_DIR = r'/data/archive/'

# Datastream info
DATASTREAM_NAME = 'sondeadjust'
DATA_LEVEL = 'c1'
LOCATIONS = [{'end_date': '2011-04-24', 'facility': 'M1', 'site': 'sbs', 'start_date': '2010-11-08'}, {'end_date': '2012-02-09', 'facility': 'M1', 'site': 'gan', 'start_date': '2011-09-17'}, {'end_date': '2011-01-05', 'facility': 'M1', 'site': 'grw', 'start_date': '2009-04-16'}, {'end_date': '2007-01-08', 'facility': 'M1', 'site': 'nim', 'start_date': '2006-01-07'}, {'end_date': '2008-12-28', 'facility': 'M1', 'site': 'hfe', 'start_date': '2008-05-14'}, {'end_date': '2008-01-01', 'facility': 'M1', 'site': 'fkb', 'start_date': '2007-03-24'}, {'end_date': '2012-07-16', 'facility': 'C1', 'site': 'nsa', 'start_date': '2002-04-28'}, {'end_date': '2013-06-29', 'facility': 'M1', 'site': 'pvc', 'start_date': '2012-06-25'}, {'end_date': '2005-09-15', 'facility': 'M1', 'site': 'pye', 'start_date': '2005-02-25'}, {'end_date': '2012-03-31', 'facility': 'M1', 'site': 'pgh', 'start_date': '2011-06-15'}, {'end_date': '2012-09-01', 'facility': 'C1', 'site': 'sgp', 'start_date': '1999-07-20'}, {'end_date': '2011-09-09', 'facility': 'C1', 'site': 'twp', 'start_date': '1997-08-28'}, {'end_date': '2012-04-05', 'facility': 'C2', 'site': 'twp', 'start_date': '1998-11-05'}, {'end_date': '2012-12-19', 'facility': 'C3', 'site': 'twp', 'start_date': '2002-04-01'}]

## Define site, facility, and date ranges

In [ ]:
print("The following locations and date ranges are available for this VAP:")
display(pd.DataFrame(LOCATIONS, columns=['site', 'facility', 'start_date', 'end_date']))

In [ ]:
# Define site, facility, and date range below:
site_facility = ( 'sgp', 'C1' )

# Date range. Format: YYYY-MM-DD
date_start = '2012-08-26'
date_end = '2012-08-27'

## Load data files
Loads data files from /data/archive/

In [ ]:
# Compile list of files
site, facility = site_facility
d_date_start = datetime.strptime(date_start, '%Y-%m-%d')
d_date_end = datetime.strptime(date_end, '%Y-%m-%d')
dir_path = os.path.join(DATA_DIR + site, site + DATASTREAM_NAME + facility + r'.' + DATA_LEVEL )
files_list  = []
for f in os.listdir(dir_path):
    file_date_str = f.split(r'.')[2]
    try:
        file_date = datetime.strptime(file_date_str, '%Y%m%d')
    except Exception:
        continue 
    if d_date_start <= file_date and d_date_end > file_date:
        files_list.append(os.path.join(dir_path, f))

# Load files as a single dataset
ds = act.io.armfiles.read_netcdf(files_list)
ds.clean.cleanup()
print(f'{len(files_list)} files loaded')
ds


## Plot time series data

In [ ]:
# Define the list of variables to be plotted
variables_to_plot = ['pres', 'tdry', 'dp']

In [ ]:
ts_display = act.plotting.TimeSeriesDisplay(ds)
ts_display.add_subplots((len(variables_to_plot),), figsize = (13,4*len(variables_to_plot)))

for i,v in enumerate(variables_to_plot):
    ts_ax = ts_display.plot(v, subplot_index=(i,), set_title=ds.variables[v].attrs['long_name'],linestyle='None')
    ts_ax.grid()

plt.show()


## Quality check plots

In [ ]:
# Define variable for QC plot
qc_variable = 'pres'

In [ ]:
if ('qc_' + qc_variable) in ds.variables:

    # Plot
    qc_display = act.plotting.TimeSeriesDisplay(ds)
    qc_display.add_subplots((2,), figsize = (13,10))
    qc_ax = qc_display.plot(qc_variable, subplot_index=(0,), set_title="QC results on field: " + ds.variables[qc_variable].attrs['long_name'],linestyle='None')
    qc_ax.grid()
    qc_display.qc_flag_block_plot(qc_variable, subplot_index=(1,))

    plt.show()
else:
    print(f'QC not available for the selected field: {qc_variable}')


## Field selection dropdown menu

In [ ]:
plt.ioff()

# populate dropdown menu with available variables 
available_variables = [v for v in ds.variables if not('time' in v or v.startswith('qc_') or v.startswith('source_')) and 'long_name' in ds.variables[v].attrs]
d_variable = 'pres'
dropdown = widgets.Dropdown(
    options = [(ds.variables[v].attrs['long_name'], v) for v in available_variables],
    value= d_variable,
    description='Field:',
    disabled=False,
)
dropdown.layout.margin = '0px 30% 0px 20%'
dropdown.layout.width = '50%'

# set up display
i_display = act.plotting.TimeSeriesDisplay(ds)
i_display.add_subplots((1,), figsize = (13,5))
i_ax = i_display.plot(d_variable, subplot_index=(0,), set_title=ds.variables[d_variable].attrs['long_name'],linestyle='None')
i_ax.grid()
i_fig = i_display.fig

# update plot callback function
def update_plot(change):
    i_ax.cla()
    i_ax_new = i_display.plot(change.new, subplot_index=(0,), set_title=ds.variables[change.new].attrs['long_name'],linestyle='None')
    i_ax_new.grid()
    i_fig.canvas.draw()
    i_fig.canvas.flush_events()

dropdown.observe(update_plot, names='value')

widgets.AppLayout(
    header=dropdown,
    center=i_fig.canvas,
    pane_heights=[1, 6,1]
)


## Skew-T Plot

In [ ]:
launch_times = [str(datetime.strptime(''.join(f.split('.')[-3:-1]), '%Y%m%d%H%M%S')) for f in files_list]
print('Available sonde launch times:')
display(pd.DataFrame(launch_times, columns=['Launch Time']))


In [ ]:
# select sonde launch time from the list
launch_time_index = 0
sonde_file = files_list[launch_time_index]
sonde_ds = act.io.armfiles.read_netcdf(sonde_file)

# Calculate stability indicies
sonde_ds = act.retrievals.calculate_stability_indicies(
    sonde_ds, temp_name='tdry', td_name='dp', p_name='pres', rh_name='rh'
)

# Set up plot
skewt = act.plotting.SkewTDisplay(sonde_ds, figsize=(7, 10))

# Add data
skewt.plot_from_u_and_v('u_wind', 'v_wind', 'pres', 'tdry', 'dp', set_title=f'Skew-T Plot for {launch_times[launch_time_index]}')
sonde_ds.close()
plt.show()